<a href="https://colab.research.google.com/github/IbHansen/wb-debt-simulation/blob/main/simulation/Specification_debt_simulation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import modeljupytermagic

ModuleNotFoundError: No module named 'modeljupytermagic'

In [ ]:
import pandas as pd
import re

from modelclass import model
import modelmanipulation as mp
from modelmanipulation import doable, explode

from modelpattern import list_extract

# About lists

**lists** and **sublists** are central to making ModelFlow scalable and expressive.  
They allow the modeler to define **groups of entities** (such as bonds, banks, portfolios, sectors, or countries) and to attach **attributes** or **hierarchies** to those entities.  
A list definition typically takes the following form

$list\: \underbrace{issued}_{List name} = \underbrace{issued}_{sub listname} : issued\_2025 * issued\_2050 / \\
\underbrace{issued\_year}_{sub listname} : 2025 * 2050$

Please note:
 - the list name and the first sub list name should be the same
 - case don't matter
 - a * combined with identifiers ending in digits will construct multiple list content

  `issued_2025 * issued_2050` becomes `issued_2025 issued_2026 … issued_2050`


This enables equations to be written once as templates and then **expanded automatically** across all relevant list members.  

Sublists can be used to define logical relationships between entities — for example, linking each bond to its home maturity, or to define the year for each issurance year.  
These structures make it possible to organize models systematically and to maintain consistency across entities that share the same behavioral equations.  

This approach eliminates redundancy and manual repetition: instead of writing dozens or hundreds of similar equations, the modeler defines a **single equation pattern**, and ModelFlow expands it dynamically over the members of a list.  

In [ ]:
%%mexplode port segment=listport
# List definitions


> list issued = issued        : issued_2025 * issued_2050 /
>>              issued_year   :        2025 *        2050

>list bond = bond :     1_year_dom 5_year_dom 10_year_dom  1_year_usd 5_year_usd 10_year_usd /
>>        maturity :              1          5          10           1          5          10 /
>>        currency :            dom        dom         dom         usd        usd         usd /
>>         domestic :             1          1           1           0          0           0 /
>>         foreign :              0          0           0           1          1           1

In [ ]:
port_replacements=[('__bond','__{bond}__{issued}'),
                   ('fx_rate','((dom_{currency}>0.000001) + (dom_{currency} < 0.000001))')
                  ]

In [ ]:
%%mexplode port segment=logical replacements=port_replacements

## Define logical variables
The folowing variables are defined for each bond and each issuing year. They are **logical variables** that is they can be either 0.0 or 1.0.

The terms enclosed in `{}` are taken from the relevant sublists. the  `current_year` are taken from the input dataset.

> doable logical_issuing__bond = current_year == {issued_year}
> doable logical_maturing__bond =  current_year == {issued_year}+{maturity}

> doable logical_paying_years__bond =  {issued_year} < current_year <= ({issued_year} + {maturity})



In [ ]:
%%mexplode port segment=core replacements=port_replacements

## Funding need
>funding_need = deficit+amortizing__all + interest_payments__all

## Interest rates
The first year after the issurance the interest rate is taken from the interest rate of the relevant maturity.

In the rest of the paying years the interest rate are set to the bonds interest rate the previous year
> doable [bond=DOMESTIC] interest_rate__bond = interest_rate__{maturity}(-1)            * logical_issuing__bond (-1) + interest_rate__bond(-1) * logical_paying_years__bond
> doable [bond=foreign]  interest_rate__bond = interest_rate_{currency}__{maturity}(-1) * logical_issuing__bond (-1) + interest_rate__bond(-1) * logical_paying_years__bond


## in_currency

### issue in the individual bonds
> doable new_issue_in_currency__bond = new_issue_share__{bond}/100 * funding_need * logical_issuing__bond  /  fx_rate

### Amortizing
> doable  amortizing_in_currency__bond = stock_primo_in_currency__bond * logical_maturing__bond


### End of year stock
> doable stock_ultimo_in_currency__bond = stock_primo_in_currency__bond+ new_issue_in_currency__bond-amortizing_in_currency__bond

### Start of year stock
> doable stock_primo_in_currency__bond = stock_ultimo_in_currency__bond(-1)

### Interest rate payment
> doable interest_in_currency__bond = stock_primo_in_currency__bond * interest_rate__bond/100

## Domestic
### Interest rate payment
> doable <sum=all>  interest_payments__bond    = interest_in_currency__bond  * fx_rate

### Amortizing

> doable <sum=all>  amortizing__bond = amortizing_in_currency__bond * fx_rate

### Stock
> doable <sum=all> stock_primo__bond = stock_primo_in_currency__bond * fx_rate
> doable <sum=all> stock_ultimo__bond = stock_ultimo_in_currency__bond * fx_rate



In [ ]:
port_dict.keys()

In [ ]:
%%mexplode port replacements=port_replacements
kk

In [ ]:
years = [y for y in range(2025,2050,1)]
dfstart = pd.DataFrame(years,index=years,columns=['CURRENT_YEAR'])
dfstart

In [ ]:
df = dfstart.upd('''
new_issue_share__1_year_dom = 40
new_issue_share__5_year_dom  = 40
new_issue_share__10_year_dom  = 20
interest_rate__1 = 1
interest_rate__5 = 1
interest_rate__10 = 1
interest_rate_usd__1 = 3
interest_rate_usd__5 = 3
interest_rate_usd__10 = 3
<2026> deficit  = 100

''')
df.head()


In [ ]:
df.loc[:,mport.vlist('interest_rate__*')] = 1

In [ ]:
res = mport(df,silent=1)

In [ ]:
mport['INTEREST_RATE__*__*26  interest*__*26 funding* paying_years__*26'].df.T


In [ ]:
mport['stock_primo__all stock_ultimo__all interest*__*26 funding* paying_years__*26'].df.T


In [ ]:
mport['interest_payments__all interest_in_currency__*202? interest_rate__*202?'].df.head().T

In [ ]:
mport.NEW_ISSUE_IN_CURRENCY__1_YEAR_DOM__ISSUED_2026

In [ ]:
mport.modeldump('model/port')

In [ ]:
mport['logical_pay*__5*'].df.T